# Construct Datasets for benchmarking tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-10-08  

## Dataset1. Enzyme None-enzyme Dataset
The enzyme dataset is consists of two parts: a training set and a testing set. The training set is from snapshot Feb-2018 and excludes those deleted items in snapshot Jun-2020. The training set is consists of 467,973 records, of which 222,290 are enzymes, and 245,683 are not enzymes. The testing set is from snapshot Jun-2020 and excludes these items that appeared in snapshot Feb-2018. The testing set is consists of 8033 records, of which 3579 are enzymes, and 4454 are not-enzymes. Unlike previous works,  we did not filter any sequences in terms of length and homology to make the data more inclusive. We make a label for each sequence, 1 for enzyme and 0 for not-enzyme. 

## Dataset2. Enzyme Quantity Dataset
The enzyme quantity dataset only contains enzyme data, contain 13,108 records. The function quantity ranges from 2 to 10.

## Dataset 3: EC number Dataset

Similar to the enzyme quantity dataset, the EC number dataset is consists of 225,221 enzyme records, 221,642 are training-set, and the rest 3579 are testing-set, covering 4852 EC numbers. Up to Feb 2020, there still exist 267 EC numbers that the model can not handle in the benchmarking. Thus, we exclude the sequences with these 267 EC numbers in the evaluation process. But, this problem can be resolved in the production scenario because we use the entire data from Swiss-Prot. Now the EC coverage is 5307 and can be automatically extended, for the training is real-time based on the publication of Swiss-Prot every 8 weeks. 

## 1. Import packages

In [95]:
import numpy as np
import pandas as pd

import os
import time
import datetime
import sys
from tqdm import tqdm
import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import exact_ec_from_uniprot as exactec
import minitools as mtool

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库


%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Define Functions

In [2]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel

## 3. Download rawdata from unisprot

In [3]:
# download location ./tmp
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/previous_major_releases/release-2018_02/knowledgebase/uniprot_sprot-only2018_02.tar.gz -o ./tmp/uniprot_sprot-only2018_02.tar.gz -q -c
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/previous_major_releases/release-2020_06/knowledgebase/uniprot_sprot-only2020_06.tar.gz -o ./tmp/uniprot_sprot-only2020_06.tar.gz -q -c

No state file, cannot resume!
No state file, cannot resume!


## 4. Extract records from rawdata

In [18]:
# 2018 data
! tar -zxvf ./tmp/uniprot_sprot-only2018_02.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2018.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

# 2020 data
! tar -zxvf ./tmp/uniprot_sprot-only2020_06.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2020.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz
uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz


In [27]:
exactec.run_exact_task(infile=cfg.TEMPDIR+'sprot2018.data.gz', outfile=cfg.DATADIR+'sprot2018.tsv')
exactec.run_exact_task(infile=cfg.TEMPDIR+'sprot2020.data.gz', outfile=cfg.DATADIR+'sprot2020.tsv')

556825it [03:41, 2516.40it/s]

finished use time 221.339 s


## 5. Load records

In [39]:
#加载数据并转换时间格式
sprot2018 = pd.read_csv(cfg.DATADIR+'sprot2018.tsv', sep='\t',header=0) #读入文件
sprot2018 = mtool.convert_DF_dateTime(inputdf = sprot2018)


sprot2020 = pd.read_csv(cfg.DATADIR+'sprot2020.tsv', sep='\t',header=0) #读入文件
sprot2020 = mtool.convert_DF_dateTime(inputdf = sprot2020)


## 6. Preprocessing
### 6.1 Drop Duplicates

In [51]:
sprot2018.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2018.reset_index(drop=True, inplace=True)
sprot2020.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2020.reset_index(drop=True, inplace=True)

### 6.3 Split Tain Test

In [87]:
train = sprot2018.iloc[:,np.r_[0,2:8,10:12]]
test = sprot2020.iloc[:,np.r_[0,2:8,10:12]]
test =test[~test.seq.isin(train.seq)]
test.reset_index(drop=True, inplace=True)

### 6.4 Trim string

In [97]:
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim
    
    test.ec_number = test.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test.seq = test.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

In [98]:
train

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
0,Q3J1A3,False,False,0,-,0,1986-07-21,MADKSDLGYTGLTDEQAQELHSVYMSGLWLFSAVAIVAHLAVYIWRPWF,49
1,P02157,False,False,0,-,0,1986-07-21,MGLSDGEWQLVLNVWGKVEADLAGHGQEVLIRLFKGHPETLEKFDK...,154
2,P02178,False,False,0,-,0,1986-07-21,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,154
3,P02194,False,False,0,-,0,1986-07-21,MGLSDGEWQLVLNIWGKVETDEGGHGKDVLIRLFKGHPETLEKFDK...,154
4,P01915,False,False,0,-,0,1986-07-21,MVWLPRVPCVAAVILLLTVLSPPVALVRDTRPRFLEYVTSECHFYN...,264
...,...,...,...,...,...,...,...,...,...
469129,Q21221,True,False,1,3.1.2.22,4,2018-02-28,MSSGAPSGSSMSSTPGSPPPRAGGPNSVSFKDLCCLFCCPPFPSSI...,332
469130,Q6QJ72,True,False,1,4.2.1.96,4,2018-02-28,MSRLLLPKLFSISRTQVPAASLFNNLYRRHKRFVHWTSKMSTDSVR...,187
469131,C0HL68,False,False,0,-,0,2018-02-28,GLFSKPAGKGIKNLIPKGVKHIGKEVGKDVIRTGIDVAGCKIKGEC,46
469132,C0HK74,False,False,0,-,0,2018-02-28,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...,56


In [99]:
test

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
0,P02462,False,False,0,-,0,1986-07-21,MGPRLSVWLLLLPAALLLHEEHSRAAAKGGCAGSGCGKCDCHGVKG...,1669
1,P00939,True,True,2,"5.3.1.1, 4.2.3.3",4,1986-07-21,MAPSRKFFVGGNWKMNGRKKNLGELITTLNAAKVPADTEVVCAPPT...,249
2,P02340,False,False,0,-,0,1986-07-21,MTAMEESQSDISLELPLSQETFSGLWKLLPPEDILPSPHCMDDLLL...,390
3,P01848,False,False,0,-,0,1986-07-21,IQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDK...,140
4,P01733,False,False,0,-,0,1986-07-21,MDSWTFCCVSLCILVAKHTDAGVIQSPRHEVTEMGQEVTLRCKPIS...,115
...,...,...,...,...,...,...,...,...,...
8028,A0A061ACU2,False,False,0,-,0,2020-12-02,MTVPPLLKSCVVKLLLPAALLAAAIIRPSFLSIGYVLLALVSAVLP...,2442
8029,Q8GYS8,False,False,0,-,0,2020-12-02,MKIGVVLVLLTVFVVVMSSTSVSAQSDEDECLKETGQMQLNCFPYL...,116
8030,Q6NRV0,True,False,1,2.3.2.27,4,2020-12-02,MPIRAYCTICSDFFDNARDVAAITCGHTFHQECLLQWFHSAPHRTC...,464
8031,C5DLH0,True,True,2,"3.6.1.-, 3.6.1.-",3,2020-12-02,MGLSADLFVQKRSAASSLKQPKELGFYSKTQEGQFLVNDSSKLSYY...,383


### 2.1 按年份统计sprot中酶与非酶数据

In [144]:
sprot = sprot.drop_duplicates(subset=['seq'], keep='first', inplace=False)
sprot = sprot.reset_index(drop=True)

sprot_years = sprot.groupby([sprot['date_annotation_update'].dt.year, sprot['isemzyme']]).agg({'count'}).iloc[:,0:1]
sprot_years['yearc'] = sprot_years.index
sprot_years = sprot_years.reset_index(drop = True)

sprot_years['year'] = sprot_years.yearc.apply(lambda x: str(x).replace(')', '').replace('(','').split(',')[0].strip())
sprot_years['isEmzyme'] = sprot_years.yearc.apply(lambda x: str(x).replace(')', '').replace('(','').split(',')[1].strip().replace('\n',''))
sprot_years['value_count'] = sprot_years.iloc[:,0]
sprot_years = sprot_years[['year', 'isEmzyme', 'value_count']]
sprot_years = pd.DataFrame(np.array(sprot_years), columns=['year', 'isEmzyme', 'value_count'])
sprot_years.sort_values(by=['year'], inplace=True)

In [15]:
sprot_years.to_csv('./results/emzyme_year.csv')

In [149]:
sprot_years[sprot_years.isEmzyme=='False']

,year,isEmzyme,value_count
0,2019,False,9650
2,2020,False,108375
4,2021,False,138511


## 3. 数据去重, 并筛选大于50bp的序列

In [6]:
# 统计重复数据
table_repeat=pd.value_counts(sprot.seq, sort=True)
table_repeat = pd.DataFrame(table_repeat)
table_repeat['sequence'] = table_repeat.index
table_repeat.columns=['repeat', 'seq']
table_repeat = table_repeat.reset_index(drop=True)

#准备画图数据
figure_data = pd.value_counts(table_repeat.repeat, sort=True)
figure_data = pd.DataFrame(figure_data)
figure_data['x'] = figure_data.index
figure_data.columns=['y', 'x']

bar = (
    Bar(init_opts=opts.InitOpts(width="1700px",
                                height="750px",
                                page_title="sprot",
                                theme=ThemeType.CHALK))
    .add_xaxis(list(figure_data.x))
    .add_yaxis("重复次数", list(figure_data.y))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="SPROT重复序列数据统计"),
        datazoom_opts=opts.DataZoomOpts()
    )
)
bar.load_javascript()
bar.render_notebook()


In [179]:
# 去除重复数据, 选择 >50bp的序列
sprot = sprot.drop_duplicates(subset=['seq'], keep='first', inplace=False)
sprot = sprot.reset_index(drop=True)
base_data = sprot[sprot.seqlength >= 50]
base_data.head(2)

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P02734,ANP4_PSEAM,False,False,1,-,0,1986-07-21,1987-08-13,2019-12-11,MRITEANPDPDAKAVPAAAAPSTASDAAAAAAATAATAAAAAAATA...,85


In [6]:
dup = sprot[sprot.duplicated('seq')].sort_values(by=['seq'])
dup.to_csv('./data/sprot_duplicated.tsv', sep='\t')

## 4. 序列比对,对非酶数据选取同源率 <40% 的序列

In [195]:
emzyme_data = base_data[base_data.isemzyme == 'True']
noemzyme_data = base_data[base_data.isemzyme == 'False']
res_data = funclib.getblast(emzyme_data, noemzyme_data)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [208]:
base_data

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P02734,ANP4_PSEAM,False,False,1,-,0,1986-07-21,1987-08-13,2019-12-11,MRITEANPDPDAKAVPAAAAPSTASDAAAAAAATAATAAAAAAATA...,85
2,P00484,CAT3_ECOLX,True,False,1,2.3.1.28,4,1986-07-21,1988-08-01,2021-04-07,MNYTKFDVKNWVRREHFEFYRHRLPCGFSLTSKIDITTLKKSLDDS...,213
3,P00486,CAT2_STAAU,True,False,1,2.3.1.28,4,1986-07-21,1986-07-21,2021-04-07,MTFNIIKLENWDRKEYFEHYFNQQTTYSITKEIDITLFKDMIKKKG...,215
4,P60175,TPIS_PANTR,True,True,2,"5.3.1.1, 4.2.3.3",4,1986-07-21,2007-01-23,2020-12-02,MAPSRKFFVGGNWKMNGRKQSLGELIGTLNAAKVPADTEVVCAPPT...,249
...,...,...,...,...,...,...,...,...,...,...,...,...
454611,Q18490,TOST1_CAEEL,False,False,1,-,0,2021-04-07,1996-11-01,2021-04-07,MTTAIQYQVIPLADREHSNKVASILENKYKTATNKRITLNERFGVL...,142
454612,A0A1X9WEP1,NCMP_SACSY,True,False,1,2.1.1.351,4,2021-04-07,2017-08-30,2021-04-07,MVFDRLAGIYDATGVEFFRPVARRLLDLVDPRPGVDLLDVGCGRGA...,280
454613,Q6GN86,IDLC_XENLA,False,False,1,-,0,2021-04-07,2004-07-19,2021-04-07,MIPPADSLLKHDNPVLISKNTERKSPKSRPLKVSSPQTVLTAPVPP...,254
454614,K4GKX2,73C10_BARVU,True,False,1,2.4.1.368,4,2021-04-07,2013-01-09,2021-04-07,MVSEITHKSYPLHFVLFPFMAQGHMIPMVDIARLLAQRGVKITIVT...,495


In [212]:
em_nem_data=base_data[base_data.isemzyme!='uncertain']

In [213]:
# 按阈值筛选
notinlist = list(res_data[res_data.pident>40].id)

# 过滤同源性高的序列
base_data = base_data[~base_data.id.isin(notinlist)]
base_data = base_data.reset_index(drop=True)
em_nem_data=base_data[base_data.isemzyme!='uncertain']
em_nem_data.reset_index(drop=True, inplace=True)

## 5. 保存文件

In [214]:
h5 = pd.HDFStore('./data/emzyme_noemzyme_uncertain.h5', 'w', complevel=4, complib='blosc')
h5['data'] = base_data
h5.close()

h5 = pd.HDFStore('./data/emzyme_noemzyme.h5', 'w', complevel=4, complib='blosc')
h5['data'] = base_data
h5.close()

## 6. 划分训练集、测试集

In [240]:
emzyme_noemzyme_data=pd.read_hdf('./data/emzyme_noemzyme_with_unirep.h5',key='data')
emzyme_noemzyme_head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
emzyme_noemzyme_data.columns = emzyme_noemzyme_head

In [241]:
t_thres = datetime.datetime(2009, 12, 14, 0, 0)

#训练集
train = emzyme_noemzyme_data[emzyme_noemzyme_data.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = emzyme_noemzyme_data[emzyme_noemzyme_data.date_integraged > t_thres ].sort_values(by='date_integraged')

# 训练集
h5 = pd.HDFStore('./tasks/task1/data/train.h5', 'w', complevel=4, complib='blosc')
h5['data'] = train
h5.close()

#测试集
h5 = pd.HDFStore('./tasks/task1/data/test.h5', 'w', complevel=4, complib='blosc')
h5['data'] = test
h5.close()
# train.iloc[:,np.r_[0, 2, 7,10:12, 12:1912]]

# Task2. 预测是单功能酶还是多功能数据集构建



## 任务简介
该任务通过给定酶序列，预测该酶还是多功能酶还是单功能酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的酶进行单功能/多功能预测。


## 数据统计
- 遵照任务1找到的时间节点对数据进行划分
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 


|     Items    | 单功能酶       |   多功能非酶    |合计                               |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 185,453 | 13,239   | 198,692（198,692/219,227=90.63%)  |
| 测试集        | 18,868  | 25,594    | 20,535 (20,535/219,227=9.37% )  |


## 数据集构建方法

* 根据蛋白注释信息与之前划定的酶与非酶数据集，将「酶」数据进行分类。
* 有1个EC号的被定义为「单功能酶」，有多个EC号的被定义为「多功能酶」。


In [242]:
emzyme_data = emzyme_noemzyme_data[emzyme_noemzyme_data.isemzyme]

In [243]:
#训练集
train = emzyme_data[emzyme_data.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = emzyme_data[emzyme_data.date_integraged > t_thres ].sort_values(by='date_integraged')

# 训练集
h5 = pd.HDFStore('./tasks/task2/data/train.h5', 'w', complevel=4, complib='blosc')
h5['data'] = train
h5.close()

#测试集
h5 = pd.HDFStore('./tasks/task2/data/test.h5', 'w', complevel=4, complib='blosc')
h5['data'] = test
h5.close()

# Task3. 预测EC号



## 任务简介
该任务通过给定酶序列，预测该酶的反应类型（通过EC号表示）。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的酶进行反应类别预测。


## 数据统计
- 遵照任务1找到的时间节点对数据进行划分
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 


|     Items    |  |      |数据条数                     |
| ------------ | --------| --------- |----------------------------------|
| 训练集        |  |      | 185,453 |
| 测试集        |  |      | 18,868  |


## 数据集构建方法

* 根据蛋白注释信息与之前划定的「酶」数据，给序列标定EC号。
* 未根据样本数量对数据进行过滤。

In [244]:
emzyme_noemzyme_data=pd.read_hdf('./data/emzyme_noemzyme_with_unirep.h5',key='data')
emzyme_noemzyme_head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
emzyme_noemzyme_data.columns = emzyme_noemzyme_head

#酶
emzyme_data = emzyme_noemzyme_data[emzyme_noemzyme_data.isemzyme]
#单功能酶
emzyme_single_data = emzyme_data[~emzyme_data.isMultiFunctional].reset_index(drop=True)

#标签、标签字典
label_code = set(emzyme_single_data.ec_number)
label_dict = {v: k for k,v in zip( range(len(label_code)), label_code)} 
emzyme_single_data['ec_label'] = emzyme_single_data.apply(lambda x: label_dict.get(x['ec_number']), axis=1)

# EC 号出现频率统计
countsvlues = pd.DataFrame(emzyme_single_data['ec_number'].value_counts())
countsvlues = countsvlues.rename(columns={'ec_number': 'valuecounts'})
countsvlues['ec_number'] = countsvlues.index
countsvlues=countsvlues.reset_index(drop=True)
label_appear_dict = {v: k for v,k in zip(countsvlues.ec_number, countsvlues.valuecounts )} 
emzyme_single_data['ec_appears'] = emzyme_single_data.apply(lambda x: label_appear_dict.get(x['ec_number']), axis=1)
emzyme_single_data.head(5)

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,...,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900,ec_label,ec_appears
0,P00693,AMY1_HORVU,True,False,1,3.2.1.1,4,1986-07-21,1986-07-21,2021-04-07,...,0.091042,0.331697,-0.060142,-0.067148,-0.039508,-0.044459,-0.043506,0.125468,2563,118
1,P00687,AMY1_MOUSE,True,False,1,3.2.1.1,4,1986-07-21,2011-07-27,2021-04-07,...,0.096705,0.355171,-0.057752,-0.395335,-0.078417,0.063923,0.026447,0.074622,2563,118
2,P03116,VE1_BPV1,True,False,1,3.6.4.12,4,1986-07-21,1986-07-21,2021-04-07,...,0.013461,-0.370580,-0.049517,0.044590,-0.005917,0.222023,-0.105520,0.339802,512,1843
3,P03112,VE1_CRPVK,True,False,1,3.6.4.12,4,1986-07-21,1986-07-21,2020-12-02,...,-0.080227,-0.700049,-0.141822,0.001246,-0.141650,0.009197,-0.067992,0.015307,512,1843
4,P03113,VE1_HPV6B,True,False,1,3.6.4.12,4,1986-07-21,1986-07-21,2020-12-02,...,-0.126759,-0.883160,-0.154552,0.000484,0.000829,0.019410,-0.056065,-0.010911,512,1843


In [ ]:
t_thres = datetime.datetime(2009, 12, 14, 0, 0)

#训练集
train = emzyme_single_data[emzyme_single_data.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = emzyme_single_data[emzyme_single_data.date_integraged > t_thres ].sort_values(by='date_integraged')

# 训练集
h5 = pd.HDFStore('./tasks/task3/data/train.h5', 'w', complevel=4, complib='blosc')
h5['data'] = train
h5.close()

#测试集
h5 = pd.HDFStore('./tasks/task3/data/test.h5', 'w', complevel=4, complib='blosc')
h5['data'] = test
h5.close()